# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 2.0 - Panel Integration  
**特性**: 可导出静态 HTML（控件 + 图表都可交互）

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成基本 AI Context** - 数据结构信息
4. **生成完整 AI Prompt** - 包含所有文档（复制给 AI）
5. **选择维度** - 选择分析维度
6. **创建 Panel 仪表盘** - AI 生成分析逻辑
7. **导出 HTML** - 分享给他人

---


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
# ========================================
# 移除 JupyterLab 宽度限制（关键！）
# ========================================
from IPython.display import HTML, display

display(HTML("""
<style>
    /* 核心：解除 JupyterLab 4 的宽度限制 */
    .jp-Notebook { 
        --jp-notebook-max-width: 100% !important; 
    }
    
    /* 确保所有输出容器占满宽度 */
    .jp-Notebook-cell, 
    .jp-Cell-outputWrapper, 
    .jp-OutputArea-output, 
    .jp-OutputArea-child { 
        max-width: none !important; 
        width: 100% !important; 
    }
    
    /* Panel 根容器强制铺满 */
    .bk-root, .bk-root > .bk { 
        width: 100% !important; 
        max-width: none !important; 
    }
</style>
"""))

print("✅ JupyterLab 宽度限制已移除")

# ========================================
# 加载框架
# ========================================
import polars as pl
import plotly.express as px
import panel as pn

from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 2.0 - Panel Integration")


✅ JupyterLab 宽度限制已移除


✅ 环境初始化完成
📚 框架版本: Phase 2.0 - Panel Integration


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行 × {df_df.width} 列")
print(f"📊 数据预览:\n")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行 × 69 列
📊 数据预览:



|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 🤖 Step 3: 生成基本 AI Context

生成数据概览

In [3]:
# 生成基本 AI Context
print("📋 数据结构信息:\n")
print("=" * 80)
print(session.get_ai_context())
print("=" * 80)

print("\n💡 提示: 下一步会生成完整的 AI Prompt")


📋 数据结构信息:

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float64)
- `溢额未决` (Float64

## 📋 Step 4: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

这个 cell 会输出：
- 数据结构信息
- 完整的 Panel Dashboard 使用指南
- 代码模板
- 常见错误对比
- 检查清单


In [4]:
# ========================================
# 📋 给 AI 的完整信息（复制全部输出给 AI）
# ========================================

# 直接执行最新的 Step 4 模板
import os

# 自动找到项目根目录
def find_project_root():
    """向上查找包含 src/ 和 notebooks/templates/ 的项目根目录"""
    current = os.path.abspath('.')
    while current != '/':
        # 检查是否存在项目标志
        if (os.path.exists(os.path.join(current, 'src')) and 
            os.path.exists(os.path.join(current, 'notebooks', 'templates'))):
            return current
        # 向上一级
        current = os.path.dirname(current)
    # 找不到就返回当前目录
    return os.path.abspath('.')

project_root = find_project_root()
step4_path = os.path.join(project_root, 'notebooks', 'templates', 'step4_standalone.py')

# 调试信息（可选，帮助诊断）
# print(f"项目根目录: {project_root}")
# print(f"Step 4 路径: {step4_path}")
# print(f"文件存在: {os.path.exists(step4_path)}")

if os.path.exists(step4_path):
    exec(open(step4_path).read())
else:
    # 如果文件找不到，使用嵌入版本
    print("=" * 80)
    print("📋 **复制以下所有内容给 AI**")
    print("=" * 80)
    print()
    print("## 📊 数据结构")
    print()
    print(session.get_ai_context())
    print()
    print("=" * 80)
    print("## 📚 Panel Dashboard 完整使用指南")
    print("=" * 80)
    print()
    print("⚠️ 规则 0: 禁止硬编码任何维度！")
    print("必须使用: group_col = values.get('_aggregation_dimension')")
    print("必须跳过: if dim == '_aggregation_dimension': continue")
    print()
    print("详细说明见: notebooks/templates/step4_standalone.py")
    print("=" * 80)


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 5: 选择分析维度

选择你想要分析的维度字段

In [5]:
# 👤 选择你要分析的维度
# 修改下面的列表，添加或删除维度

selected_dimensions = [
    '业务年度',
    '业务险种',
    # '机构名称',  # 取消注释以启用
    # '境内境外',  # 取消注释以启用
]

print(f"✅ 已选择 {len(selected_dimensions)} 个维度:")
for dim in selected_dimensions:
    if dim in df_df.columns:
        n_unique = df_df[dim].n_unique()
        print(f"  - {dim} ({n_unique:,} 个唯一值)")
    else:
        print(f"  ⚠️  {dim} - 不存在于数据中")

print("\n💡 下一步: 创建 Panel 仪表盘")


✅ 已选择 2 个维度:
  - 业务年度 (25 个唯一值)
  - 业务险种 (26 个唯一值)

💡 下一步: 创建 Panel 仪表盘


## 📊 Step 6: 创建 Panel 仪表盘

自动创建仪表盘控件，然后让 AI 生成分析逻辑

In [6]:
# 创建 Panel 仪表盘（自动生成控件）
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘"
)

print("\n💡 下一步: 让 AI 生成分析逻辑")
print("\n📝 给 AI 的提示:")
print("="  * 80)
print("请使用 Panel Dashboard 生成分析代码。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]")
print("="  * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (2 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (3 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



💡 下一步: 让 AI 生成分析逻辑

📝 给 AI 的提示:
请使用 Panel Dashboard 生成分析代码。
已选择维度: ['业务年度', '业务险种']
需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]


## 🤖 Step 7: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

AI 应该生成：
1. `@pn.depends` 装饰的更新函数
2. 数据过滤和聚合逻辑
3. Plotly 图表
4. `dashboard.show()` 和 `dashboard.save()`


In [13]:
# ========================================
# 优化版自留额调整模拟测算（向量化计算）
# ========================================

def simulate_retention_increase_optimized():
    """
    优化版：使用向量化操作提高性能
    """
    import numpy as np
    
    # 1. 筛选最近十年数据
    current_year = 2024
    ten_years_ago = current_year - 10
    
    # 筛选最近十年数据
    recent_data = df_df.filter(
        pl.col('业务年度').cast(pl.Int32).is_between(ten_years_ago, current_year)
    )
    
    print(f"📊 数据筛选结果：")
    print(f"原始数据量：{df_df.height:,} 条")
    print(f"最近十年数据量：{recent_data.height:,} 条")
    print(f"数据期间：{recent_data['业务年度'].min()} 年 - {recent_data['业务年度'].max()} 年")
    print()
    
    # 2. 定义四种情景（单位：万元）
    scenarios = [
        {'名称': '2.25亿', '阈值': 22500},
        {'名称': '2.5亿', '阈值': 25000},
        {'名称': '2.75亿', '阈值': 27500},
        {'名称': '3亿', '阈值': 30000}
    ]
    
    # 3. 当前基准情景（2亿上限） - 使用原始数据
    print("计算当前基准情景...")
    current_result = calculate_profit_vectorized(recent_data, None)
    
    # 4. 计算各情景结果（批量处理）
    scenario_results = []
    
    for scenario in scenarios:
        print(f"正在计算情景：自留额提高到 {scenario['名称']}...")
        result = calculate_scenario_profit_vectorized(recent_data, scenario['阈值'])
        scenario_results.append(result)
    
    # 5. 生成对比分析
    print("\n" + "="*80)
    print("📈 自留额调整模拟测算结果对比")
    print("="*80)
    
    display_comparison_table(current_result, scenario_results, scenarios)
    
    # 6. 可视化展示
    create_visualizations(current_result, scenario_results, scenarios)
    
    return {
        'current': current_result,
        'scenarios': dict(zip([s['名称'] for s in scenarios], scenario_results))
    }


def calculate_scenario_profit_vectorized(data, new_threshold):
    """
    向量化计算单个新自留额门槛下的净自留利润
    """
    # 将所有金额转换为万元（向量化操作）
    data_wan = data.with_columns([
        (pl.col('总保额') / 10000).alias('总保额_万'),
        (pl.col('自留保额') / 10000).alias('自留保额_万'),
        (pl.col('附加自留保额') / 10000).alias('附加自留保额_万'),
        (pl.col('临分分保额') / 10000).alias('临分分保额_万'),
        (pl.col('溢额分保额') / 10000).alias('溢额分保额_万'),
        (pl.col('总保费') / 10000).alias('总保费_万'),
        (pl.col('临分分保费') / 10000).alias('临分分保费_万'),
        (pl.col('溢额分保费') / 10000).alias('溢额分保费_万'),
        (pl.col('临分摊回费用') / 10000).alias('临分摊回费用_万'),
        (pl.col('溢额摊回费用') / 10000).alias('溢额摊回费用_万'),
        (pl.col('临分已决') / 10000).alias('临分已决_万'),
        (pl.col('溢额已决') / 10000).alias('溢额已决_万'),
        (pl.col('临分未决') / 10000).alias('临分未决_万'),
        (pl.col('溢额未决') / 10000).alias('溢额未决_万')
    ])
    
    # 规则1：检查总保额与新门槛关系
    condition_total_less = pl.col('总保额_万') <= new_threshold
    
    # 规则2：计算新分出金额
    new_ceded_amount = new_threshold - pl.col('自留保额_万')
    
    # 规则3：检查4.5亿上限
    total_retention = pl.col('自留保额_万') + pl.col('附加自留保额_万')
    condition_over_450 = (new_threshold + pl.col('附加自留保额_万')) > 45000
    excess_amount = pl.max_horizontal(
        (new_threshold + pl.col('附加自留保额_万') - 45000),
        0
    )
    
    # 最终需要扣减的金额
    total_ceded_needed = pl.when(condition_over_450)\
                          .then(new_ceded_amount + excess_amount)\
                          .otherwise(new_ceded_amount)\
                          .clip(lower_bound=0)  # 确保非负
    
    # 计算扣减比例（向量化）
    total_original_ceded = pl.col('临分分保额_万') + pl.col('溢额分保额_万')
    
    # 情况1：全部自留（总保额≤新门槛）
    fac_ratio_all_retained = pl.when(condition_total_less).then(0.0)
    surplus_ratio_all_retained = pl.when(condition_total_less).then(0.0)
    
    # 情况2：需要扣减
    # 先扣减临分
    fac_reduction = pl.min_horizontal(total_ceded_needed, pl.col('临分分保额_万'))
    fac_ratio = pl.when(pl.col('临分分保额_万') > 0)\
                  .then((pl.col('临分分保额_万') - fac_reduction) / pl.col('临分分保额_万'))\
                  .otherwise(0.0)
    
    # 剩余扣减溢额
    remaining_reduction = pl.max_horizontal(total_ceded_needed - fac_reduction, 0)
    surplus_reduction = pl.min_horizontal(remaining_reduction, pl.col('溢额分保额_万'))
    surplus_ratio = pl.when(pl.col('溢额分保额_万') > 0)\
                     .then((pl.col('溢额分保额_万') - surplus_reduction) / pl.col('溢额分保额_万'))\
                     .otherwise(0.0)
    
    # 合并两种情况
    final_fac_ratio = pl.when(condition_total_less)\
                       .then(fac_ratio_all_retained)\
                       .otherwise(fac_ratio)
    
    final_surplus_ratio = pl.when(condition_total_less)\
                           .then(surplus_ratio_all_retained)\
                           .otherwise(surplus_ratio)
    
    # 计算调整后的分保额（万元）
    adjusted_fac_amount = pl.col('临分分保额_万') * final_fac_ratio
    adjusted_surplus_amount = pl.col('溢额分保额_万') * final_surplus_ratio
    
    # 转换为原始单位（元）并创建新DataFrame
    adjusted_data = data.with_columns([
        # 调整临分相关字段
        (adjusted_fac_amount * 10000).alias('临分分保额_调整'),
        (pl.col('临分分保费') * final_fac_ratio).alias('临分分保费_调整'),
        (pl.col('临分摊回费用') * final_fac_ratio).alias('临分摊回费用_调整'),
        (pl.col('临分已决') * final_fac_ratio).alias('临分已决_调整'),
        (pl.col('临分未决') * final_fac_ratio).alias('临分未决_调整'),
        
        # 调整溢额相关字段
        (adjusted_surplus_amount * 10000).alias('溢额分保额_调整'),
        (pl.col('溢额分保费') * final_surplus_ratio).alias('溢额分保费_调整'),
        (pl.col('溢额摊回费用') * final_surplus_ratio).alias('溢额摊回费用_调整'),
        (pl.col('溢额已决') * final_surplus_ratio).alias('溢额已决_调整'),
        (pl.col('溢额未决') * final_surplus_ratio).alias('溢额未决_调整'),
        
        # 调整自留保额（增加）
        (pl.col('自留保额') + (fac_reduction + surplus_reduction) * 10000).alias('自留保额_调整')
    ])
    
    # 创建临时DataFrame用于利润计算
    temp_df = adjusted_data.with_columns([
        # 使用调整后的值替换原始值
        pl.when(pl.col('临分分保额_调整').is_not_null())
          .then(pl.col('临分分保额_调整'))
          .otherwise(pl.col('临分分保额')).alias('临分分保额_final'),
        
        pl.when(pl.col('临分分保费_调整').is_not_null())
          .then(pl.col('临分分保费_调整'))
          .otherwise(pl.col('临分分保费')).alias('临分分保费_final'),
        
        # 同样处理其他字段...
    ])
    
    # 计算调整后的净自留利润
    return calculate_profit_vectorized(temp_df, new_threshold)


def calculate_profit_vectorized(data, threshold):
    """
    向量化计算净自留利润
    """
    # 直接使用DataFrame的聚合操作
    result = data.select([
        # 总保费（亿元）
        (pl.col('总保费').sum() / 1e8).alias('total_premium'),
        
        # 分出保费（亿元）- 临分和溢额使用调整后的值
        ((pl.col('临分分保费').sum() + 
          pl.col('溢额分保费').sum() + 
          pl.col('协议分保费').sum() + 
          pl.col('成数分保费').sum() + 
          pl.col('OPENCOVER分保费').sum()) / 1e8).alias('ceded_premium'),
        
        # 总赔款（亿元）
        ((pl.col('总已决赔款').sum() + pl.col('总未决赔款').sum()) / 1e8).alias('total_claims'),
        
        # 分入费用（亿元）
        ((pl.col('分入手续费').sum() + 
          pl.col('分入出单费').sum() + 
          pl.col('分入经纪费').sum()) / 1e8).alias('inward_expenses'),
        
        # 直保手续费（亿元）
        (pl.col('手续费').sum() / 1e8).alias('direct_commission'),
        
        # 摊回费用（亿元）
        ((pl.col('临分摊回费用').sum() + 
          pl.col('溢额摊回费用').sum() + 
          pl.col('协议摊回费用').sum() + 
          pl.col('成数摊回费用').sum() + 
          pl.col('OPENCOVER摊回费用').sum()) / 1e8).alias('recovered_expenses'),
        
        # 摊回赔款（亿元）
        ((pl.col('临分已决').sum() + pl.col('临分未决').sum() +
          pl.col('溢额已决').sum() + pl.col('溢额未决').sum() +
          pl.col('协议已决').sum() + pl.col('协议未决').sum() +
          pl.col('成数已决').sum() + pl.col('成数未决').sum() +
          pl.col('OPENCOVER已决').sum() + pl.col('OPENCOVER未决').sum()) / 1e8).alias('recovered_claims'),
        
        # 自留相关（亿元）
        ((pl.col('自留保费').sum() + pl.col('附加自留保费').sum()) / 1e8).alias('retention_premium'),
        
        ((pl.col('自留已决').sum() + pl.col('附加自留已决').sum() +
          pl.col('自留未决').sum() + pl.col('附加自留未决').sum()) / 1e8).alias('retention_claims'),
        
        # 记录数
        pl.len().alias('record_count')
    ]).row(0, named=True)
    
    # 计算净自留利润（亿元）
    net_profit = (
        result['total_premium'] 
        - result['ceded_premium'] 
        - result['total_claims'] 
        - result['inward_expenses'] 
        - result['direct_commission'] 
        + result['recovered_expenses'] 
        + result['recovered_claims']
    )
    
    return {
        'threshold': f"{threshold/100:.1f}亿" if threshold else "当前(2亿)",
        'threshold_value': threshold,
        'net_profit': net_profit,
        'total_premium': result['total_premium'],
        'ceded_premium': result['ceded_premium'],
        'total_claims': result['total_claims'],
        'inward_expenses': result['inward_expenses'],
        'direct_commission': result['direct_commission'],
        'recovered_expenses': result['recovered_expenses'],
        'recovered_claims': result['recovered_claims'],
        'retention_premium': result['retention_premium'],
        'retention_claims': result['retention_claims'],
        'record_count': result['record_count']
    }


# 简化版对比表格显示
def display_comparison_table(current_result, scenario_results, scenarios):
    """显示简化对比表格"""
    import pandas as pd
    
    print("\n📊 净自留利润对比（单位：亿元）")
    print("-" * 70)
    
    # 当前基准
    print(f"当前基准（2亿上限）: {current_result['net_profit']:>10,.2f} 亿元")
    
    # 各情景
    for i, result in enumerate(scenario_results):
        profit_change = result['net_profit'] - current_result['net_profit']
        change_percent = (profit_change / abs(current_result['net_profit'])) * 100
        
        print(f"{scenarios[i]['名称']:>8}: {result['net_profit']:>10,.2f} 亿元 "
              f"(变化: {profit_change:>+8,.2f} 亿元, {change_percent:>+6.1f}%)")
    
    print("\n📈 自留相关指标变化（亿元）")
    print("-" * 70)
    print(f"{'情景':<8} {'自留保费':>12} {'自留赔款':>12} {'净自留':>12} {'分出保费':>12}")
    print("-" * 70)
    
    # 当前基准
    print(f"{'当前':<8} "
          f"{current_result['retention_premium']:>12,.2f} "
          f"{current_result['retention_claims']:>12,.2f} "
          f"{current_result['net_profit']:>12,.2f} "
          f"{current_result['ceded_premium']:>12,.2f}")
    
    # 各情景
    for i, result in enumerate(scenario_results):
        print(f"{scenarios[i]['名称']:<8} "
              f"{result['retention_premium']:>12,.2f} "
              f"{result['retention_claims']:>12,.2f} "
              f"{result['net_profit']:>12,.2f} "
              f"{result['ceded_premium']:>12,.2f}")


# 简化版可视化（快速生成）
def create_visualizations(current_result, scenario_results, scenarios):
    """创建简化可视化"""
    import plotly.graph_objects as go
    
    # 准备数据
    labels = ['当前(2亿)'] + [s['名称'] for s in scenarios]
    net_profits = [current_result['net_profit']] + [r['net_profit'] for r in scenario_results]
    retention_premiums = [current_result['retention_premium']] + [r['retention_premium'] for r in scenario_results]
    retention_claims = [current_result['retention_claims']] + [r['retention_claims'] for r in scenario_results]
    
    # 创建图表
    fig = go.Figure()
    
    # 净自留利润柱状图
    fig.add_trace(go.Bar(
        x=labels,
        y=net_profits,
        name='净自留利润',
        marker_color='#2ca02c',
        text=[f'{x:,.1f}亿' for x in net_profits],
        textposition='auto'
    ))
    
    # 自留保费线图
    fig.add_trace(go.Scatter(
        x=labels,
        y=retention_premiums,
        name='自留保费',
        mode='lines+markers+text',
        line=dict(color='#ff7f0e', width=3),
        marker=dict(size=10),
        yaxis='y2',
        text=[f'{x:,.1f}亿' for x in retention_premiums],
        textposition="top center"
    ))
    
    # 自留赔款线图
    fig.add_trace(go.Scatter(
        x=labels,
        y=retention_claims,
        name='自留赔款',
        mode='lines+markers+text',
        line=dict(color='#d62728', width=3),
        marker=dict(size=10),
        yaxis='y2',
        text=[f'{x:,.1f}亿' for x in retention_claims],
        textposition="top center"
    ))
    
    # 更新布局
    fig.update_layout(
        title='自留额调整模拟测算结果',
        xaxis_title='自留额门槛',
        yaxis=dict(
            title='净自留利润（亿元）',
            titlefont=dict(color='#2ca02c'),
            tickfont=dict(color='#2ca02c')
        ),
        yaxis2=dict(
            title='自留保费/赔款（亿元）',
            titlefont=dict(color='#ff7f0e'),
            tickfont=dict(color='#ff7f0e'),
            overlaying='y',
            side='right'
        ),
        height=500,
        showlegend=True
    )
    
    fig.show()


# 执行优化版模拟测算
print("🚀 开始自留额调整模拟测算（优化版）...")
print("="*80)
try:
    results = simulate_retention_increase_optimized()
    print("✅ 模拟测算完成！")
except Exception as e:
    print(f"❌ 计算过程中出现错误: {e}")
    print("\n尝试简化版本计算...")
    
    # 尝试简化版本
    results_simple = quick_simulation()

🚀 开始自留额调整模拟测算（优化版）...
📊 数据筛选结果：
原始数据量：776,739 条
最近十年数据量：548,387 条
数据期间：2014 年 - 2024 年

计算当前基准情景...
正在计算情景：自留额提高到 2.25亿...
❌ 计算过程中出现错误: unable to find column "临分分保额_万"; valid columns: ["业务年度", "机构代码", "机构名称", "业务险种", "保单号", "危险单位号", "支付币别", "业务来源", "是否联共保", "联共保我司占比", "地址代码", "标的地址或船名", "被保险人名称", "保险起期", "保险止期", "风险代码", "风险代码描述", "风险等级", "总保额", "总保费", "总保费_原币", "手续费", "总已决赔款", "总未决赔款", "自留保额", "自留保费", "自留已决", "自留未决", "附加自留保额", "附加自留保费", "附加自留已决", "附加自留未决", "协议分保额", "协议分保费", "协议摊回费用", "协议已决", "协议未决", "成数分保额", "成数分保费", "成数摊回费用", "成数已决", "成数未决", "溢额分保额", "溢额分保费", "溢额摊回费用", "溢额已决", "溢额未决", "OPENCOVER分保额", "OPENCOVER分保费", "OPENCOVER摊回费用", "OPENCOVER已决", "OPENCOVER未决", "临分分保额", "临分分保费", "临分摊回费用", "临分已决", "临分未决", "占用性质名称", "行业类别名称", "境内境外", "分入手续费率", "分入手续费", "分入出单费比例", "分入出单费", "分入经纪费率", "分入经纪费", "分入经纪人名称", "直保代理人名称", "归属机构"]

尝试简化版本计算...
🔍 运行极简测试版本...
计算当前基准...
计算阈值 225.0亿...
计算阈值 250.0亿...
计算阈值 275.0亿...
计算阈值 300.0亿...

📊 简版测算结果（基于1万条样本）:
--------------------------------------------------


In [12]:
# ========================================
# 极简版本（先测试逻辑）
# ========================================

def quick_simulation():
    """
    极简版本：先测试核心逻辑
    """
    print("🔍 运行极简测试版本...")
    
    # 只取少量数据测试
    sample_data = df_df.limit(10000)  # 先测试1万条
    
    # 四种情景
    scenarios = [22500, 25000, 27500, 30000]
    
    print("计算当前基准...")
    current_profit = quick_calculate_profit(sample_data)
    
    results = []
    for threshold in scenarios:
        print(f"计算阈值 {threshold/100:.1f}亿...")
        adjusted_profit = quick_calculate_adjusted_profit(sample_data, threshold)
        results.append({
            'threshold': threshold,
            'profit': adjusted_profit,
            'change': adjusted_profit - current_profit
        })
    
    # 显示结果
    print("\n📊 简版测算结果（基于1万条样本）:")
    print("-" * 50)
    print(f"当前基准（2亿上限）: {current_profit:,.2f} 亿元")
    for r in results:
        print(f"{r['threshold']/100:.1f}亿: {r['profit']:,.2f} 亿元 "
              f"(变化: {r['change']:+,.2f} 亿元)")
    
    return results


def quick_calculate_profit(data):
    """快速计算净自留利润"""
    total_premium = data['总保费'].sum() / 1e8
    ceded_premium = (
        data['临分分保费'].sum() + 
        data['溢额分保费'].sum() + 
        data['协议分保费'].sum() + 
        data['成数分保费'].sum() + 
        data['OPENCOVER分保费'].sum()
    ) / 1e8
    
    total_claims = (data['总已决赔款'].sum() + data['总未决赔款'].sum()) / 1e8
    
    inward_expenses = (
        data['分入手续费'].sum() + 
        data['分入出单费'].sum() + 
        data['分入经纪费'].sum()
    ) / 1e8
    
    direct_commission = data['手续费'].sum() / 1e8
    
    recovered_expenses = (
        data['临分摊回费用'].sum() + 
        data['溢额摊回费用'].sum() + 
        data['协议摊回费用'].sum() + 
        data['成数摊回费用'].sum() + 
        data['OPENCOVER摊回费用'].sum()
    ) / 1e8
    
    recovered_claims = (
        data['临分已决'].sum() + data['临分未决'].sum() +
        data['溢额已决'].sum() + data['溢额未决'].sum() +
        data['协议已决'].sum() + data['协议未决'].sum() +
        data['成数已决'].sum() + data['成数未决'].sum() +
        data['OPENCOVER已决'].sum() + data['OPENCOVER未决'].sum()
    ) / 1e8
    
    net_profit = (
        total_premium 
        - ceded_premium 
        - total_claims 
        - inward_expenses 
        - direct_commission 
        + recovered_expenses 
        + recovered_claims
    )
    
    return net_profit


def quick_calculate_adjusted_profit(data, threshold):
    """
    简化版调整计算
    只调整临分和溢额，使用近似方法
    """
    # 计算需要调整的比例
    # 简化：假设所有记录都需要调整，使用平均调整比例
    avg_retention = data['自留保额'].mean() / 10000
    avg_additional = data['附加自留保额'].mean() / 10000
    
    if avg_retention >= threshold:
        # 自留保额已超过新门槛，无需调整
        return quick_calculate_profit(data)
    
    # 计算平均调整比例
    total_retention = avg_retention + avg_additional
    if threshold + avg_additional > 45000:
        # 超过4.5亿上限
        adjust_ratio = 0.8  # 简化：假设调整80%
    else:
        # 简单线性调整
        adjust_ratio = max(0.5, 1 - (threshold - avg_retention) / 10000)
    
    # 创建调整后的数据（简化）
    adjusted_data = data.with_columns([
        # 调整临分和溢额
        (pl.col('临分分保费') * adjust_ratio).alias('临分分保费_adj'),
        (pl.col('溢额分保费') * adjust_ratio).alias('溢额分保费_adj'),
        (pl.col('临分摊回费用') * adjust_ratio).alias('临分摊回费用_adj'),
        (pl.col('溢额摊回费用') * adjust_ratio).alias('溢额摊回费用_adj'),
        (pl.col('临分已决') * adjust_ratio).alias('临分已决_adj'),
        (pl.col('溢额已决') * adjust_ratio).alias('溢额已决_adj'),
        (pl.col('临分未决') * adjust_ratio).alias('临分未决_adj'),
        (pl.col('溢额未决') * adjust_ratio).alias('溢额未决_adj')
    ])
    
    # 重新计算利润
    return quick_calculate_profit(adjusted_data)


# 运行简版测试
quick_results = quick_simulation()

🔍 运行极简测试版本...
计算当前基准...
计算阈值 225.0亿...
计算阈值 250.0亿...
计算阈值 275.0亿...
计算阈值 300.0亿...

📊 简版测算结果（基于1万条样本）:
--------------------------------------------------
当前基准（2亿上限）: 0.69 亿元
225.0亿: 0.69 亿元 (变化: +0.00 亿元)
250.0亿: 0.69 亿元 (变化: +0.00 亿元)
275.0亿: 0.69 亿元 (变化: +0.00 亿元)
300.0亿: 0.69 亿元 (变化: +0.00 亿元)


In [9]:
# 导出为静态 HTML
#dashboard.save("我的分析.html", embed=True)

print("✅ 已导出到: 我的分析.html")
print("💡 特性:")
print("  - 所有控件可交互")
print("  - Plotly 图表可交互")
print("  - 单个文件，可离线使用")
print("  - 可邮件分享")
print("\n🎉 分析完成！")


✅ 已导出到: 我的分析.html
💡 特性:
  - 所有控件可交互
  - Plotly 图表可交互
  - 单个文件，可离线使用
  - 可邮件分享

🎉 分析完成！
